In [3]:
import random
class DesChiffresEtDesLettres:
    def __init__(self):
        self.small_numbers = list(range(1, 11))
        self.large_numbers = [25, 50, 75, 100]
    #NUMBERS ROUND 
    def generate_target_number(self):
        #Generates a random 3-digit target number
        return random.randint(100, 999)

    def generate_six_numbers(self):
    #Combine 4 randomly selected small numbers and 2 large numbers into one list
        combined_numbers = random.sample(self.small_numbers, 4) + random.sample(self.large_numbers, 2)
    #Randomly select 6 numbers from the combined list
        selected_numbers = random.sample(combined_numbers, len(combined_numbers))
        return selected_numbers

    def evaluate_expression(self, expression, available_numbers):
        #Evaluates the player's arithmetic expression, checks if it's valid
        valid_operators = set(['+', '-', '*', '/'])
        tokens = expression.split()
        #To store the numbers and operators used in the expression
        numbers_used = []
        operators_used = []
        #Loop through the tokens to classify them as numbers or operators
        for token in tokens:
            if token.isdigit():
                numbers_used.append(int(token))  #Add valid number to list
            elif token in valid_operators:
                operators_used.append(token)  #Add valid operator to list
            else:
                #If an invalid token is found, return None indicating an invalid expression
                print(f"Invalid token in expression: {token}")
                return None
        #Validate that all numbers used come from the available numbers and aren't reused
        temp_numbers = available_numbers.copy()
        for num in numbers_used:
            if num in temp_numbers:
                temp_numbers.remove(num)  #Remove the number if it's available
            else:
                #If the number is invalid (reused or not in the available set), return None
                print(f"Invalid or reused number: {num}")
                return None
        result = self.manual_evaluate(tokens)
        if result is None:
            #If there's an error during evaluation (e.g., division by zero), return None
            print("Invalid expression or division by zero.")
        return result

    def manual_evaluate(self, tokens):
        #Manual evaluation of expression
        #Simple left-to-right evaluation
        result = int(tokens[0])  #Assume the first token is a number
        i = 1  #Start iterating from the first operator (at index 1)
        while i < len(tokens) - 1:
            operator = tokens[i]  #Current operator
            next_num = int(tokens[i + 1])  #Next number in the expression
            #Perform the corresponding operation
            if operator == '+':
                result += next_num
            elif operator == '-':
                result -= next_num
            elif operator == '*':
                result *= next_num
            elif operator == '/':
                if next_num == 0:
                    #Return None if there's a division by zero
                    return None
                result //= next_num  #Integer division
            i += 2  #Move to the next operator and number pair
        return result

    def calculate_number_round_score(self, target, guess):
        #Calculates score for the numbers round
        difference = abs(target - guess)
        #The score decreases by 1 for every 10 units of difference, with a max score of 100
        score = 100 - (difference // 10)
        return max(score, 0)  #Ensure that the score doesn't go below 0

    def find_optimal_solution(self, numbers, target):
        #Uses a brute-force approach to find the closest result to the target
        def apply_operations(nums, ops):
            #Evaluates a list of numbers and operations manually
            result = nums[0]
            for i in range(1, len(nums)):
                op = ops[i - 1]  #Get the current operator
                if op == '+':
                    result += nums[i]
                elif op == '-':
                    result -= nums[i]
                elif op == '*':
                    result *= nums[i]
                elif op == '/':
                    if nums[i] == 0:
                        return None  #Avoid division by zero
                    result = result // nums[i]  #Use integer division
            return result
        #Initialize variables to store the best result found
        best_diff = float('inf')  #Start with an infinitely large difference
        best_result = None  #No result found initially
        operators = ['+', '-', '*', '/']  #Allowed operators
        #Brute force all pairs of numbers and operators
        for num1 in numbers:
            for num2 in numbers:
                if num1 != num2:  #Ensure numbers aren't reused in pairs
                    for op in operators:
                        result = apply_operations([num1, num2], [op])
                        if result is not None:
                            diff = abs(target - result)  #Calculate how close result is to target
                            if diff < best_diff:
                                best_diff = diff
                                best_result = result
        return best_result, best_diff

    #LETTERS ROUND
    def generate_letter(self, choice):
        #Generates a random letter based on the player's choice (vowel or consonant)
        vowels = "AEIOU"
        consonants = "BCDFGHJKLMNPQRSTVWXYZ"
        if choice.lower() == 'vowel':
            #If the player chooses a vowel, randomly pick one from the vowel set
            index = random.randint(0, len(vowels) - 1)
            return vowels[index]
        else:
            #If the player chooses a consonant, randomly pick one from the consonant set
            index = random.randint(0, len(consonants) - 1)
            return consonants[index]

    def prompt_for_letters(self):
        #Prompts the player to choose 10 letters (vowel or consonant)
        letters = []
        for _ in range(10):
            choice = input("Choose 'vowel' or 'consonant': ").strip().lower()
            letter = self.generate_letter(choice)
            letters.append(letter)
            print(f"Generated letter: {letter}")
        return letters

    def get_longest_word(self, letters, word_list):
        #Finds the longest word that can be formed using the given letters
        def can_form_word(word, available_letters):
            #Helper function to check if a word can be formed from the available letters
            temp_letters = available_letters.copy()  #Copy the letter list so the original isn't modified
            for letter in word:
                if letter in temp_letters:
                    temp_letters.remove(letter)  #Remove the letter from available letters
                else:
                    return False  #If a letter is not available, the word cannot be formed
            return True

        longest_word = ""  #Store the longest valid word found
        for word in word_list:
            #Check if the word can be formed and if it is longer than the current longest word
            if can_form_word(word.upper(), letters) and len(word) > len(longest_word):
                longest_word = word  #Update the longest word if conditions are met
        return longest_word

    def calculate_word_round_score(self, longest_word_length, player_word_length):
        #Calculate score based on the length of the player's word relative to the longest possible word
        score = 100 - (longest_word_length - player_word_length) * 10
        return max(score, 0) #Ensure the score does not drop below 0

    def load_words_from_file(self, filename):
        #Loads words from a file and returns them as a list
        try:
            with open(filename, 'r') as file:
                #Read all lines from the file and strip any whitespace
                words = [line.strip() for line in file.readlines()]
            return words
        except FileNotFoundError:
            # If the file is not found, print an error message and return an empty list
            print(f"Error: The file {filename} was not found.")
            return []

game = DesChiffresEtDesLettres()
#Numbers round
target = game.generate_target_number()
numbers = game.generate_six_numbers()
print(f"Target: {target}")
print(f"Numbers: {numbers}")
#Get an expression from the user and evaluate it
expression = input("Enter an arithmetic expression using the numbers: ")
guess = game.evaluate_expression(expression, numbers)

if guess is not None:
    #Calculate and print the player's score based on the target and guess
    score = game.calculate_number_round_score(target, guess)
    print(f"Your guess: {guess}, Score: {score}")

#Letters round
letters = game.prompt_for_letters()  # Ask the player for their letter choices
print(f"Your letters: {''.join(letters)}")  # Print the selected letters
#Load word list from the file 'words.txt'
word_list = game.load_words_from_file('words.txt')
#Check if word list is loaded successfully (to avoid errors if file is missing)
if word_list:
    #Find the longest word that can be formed from the generated letters
    longest_word = game.get_longest_word(letters, word_list)
    print(f"Longest possible word: {longest_word}")
    #Get the player's word and calculate the score
    player_word = input("Enter your word: ").upper()
    score = game.calculate_word_round_score(len(longest_word), len(player_word))
    print(f"Your word: {player_word}, Score: {score}")


Target: 711
Numbers: [9, 10, 2, 3, 25, 50]
Your guess: 1250, Score: 47
Generated letter: M
Generated letter: O
Generated letter: I
Generated letter: R
Generated letter: I
Generated letter: D
Generated letter: R
Generated letter: U
Generated letter: A
Generated letter: F
Your letters: MOIRIDRUAF
Longest possible word: auriform
Your word: AURIFORM, Score: 100
